In [1]:
#Testing the CTRP-L1000-3h data by using the Achilles-L1000-120h model
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
import pickle
import scipy as sc
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns',15)
warnings.filterwarnings("ignore")

#Retrieving the data
def getData():
    #1.Reading the data
    sig_viability_info = pd.read_table(r"individual_validation\LINCS_CTRP_withoutdose_3h.txt",sep="	",dtype=str)
    sig_viability_info_filter = sig_viability_info[['signature','cell_line','drug','dose','time','cell_viability']]
    sig_GE_info = pd.read_csv(r"individual_validation\LINCS_gene_expression_3h.csv",sep=",",dtype=str)
    sig_GE_info_filter = sig_GE_info[:]

    #2.Combining the two datasets
    sig_viability = pd.merge(sig_viability_info_filter,sig_GE_info_filter,left_on='signature',right_on='cid')
    select_sig_viavility2 = pd.DataFrame(sig_viability).loc[:,'780':]
    select_sig_viavility_arr = select_sig_viavility2.copy()
    
    #3.Getting the cell viability value
    viability_value = pd.DataFrame(sig_viability).loc[:,'cell_viability']
    
    #4.Return the gene expression matrix and cell viability value
    return select_sig_viavility_arr,viability_value
    
#Retrieving the data
gene_expression,cell_viability = getData()

#Selecting the genes
selected_gene = {'332': 4.208382456402351, '4792': 4.739521929497061, '1846': 4.791617543597649, '983': 9.682334649352057, '5427': 9.890717105754407, '25803': 10.53113947309471, '10221': 10.583235087195298, '3638': 11.42185657884912, '4303': 12.416764912804702, '1514': 12.947904385899411, '6603': 19.421856578849116, '2309': 20.677242983307643, '664': 21.166470174390597, '5357': 21.63533070129589, '3486': 22.567960089062048, '2690': 22.95808771798825, '9217': 24.687426315396475, '6599': 25.958087717988242, '54386': 26.416764912804705, '5236': 29.78652587755323, '7158': 30.588326753239713, '11065': 32.5832350871953, '5050': 35.33294034878119, '3725': 35.374852630792944, '10007': 41.43203991093795, '7157': 43.692517981440886, '16': 44.11437456029001, '2146': 47.10419122820118, '991': 51.661967985174385, '29937': 52.322757016692364, '644': 56.724246931363815, '79071': 57.04700394805617, '4605': 57.578143421150884, '11182': 58.895808771798826, '2058': 63.45867719481645, '6839': 63.73443026345265, '1978': 64.56796008906205, '27244': 70.96317938403267, '1509': 72.29102806676944, '899': 77.20838245640235, '9833': 78.75088421524232, '54505': 79.12573684603525, '1759': 80.16137850834617, '481': 80.30239035251469, '3800': 83.07755394432267, '23': 85.58832675323973, '27032': 87.2021118367015, '3303': 87.8335298256094, '5982': 91.29729868647027, '22908': 91.65060569942914, '10059': 93.65178465308554, '3383': 94.18292412618027, '10180': 94.5832350871953, '11157': 97.25538640445852, '22934': 99.70388026718614, '47': 99.90599210388767, '4216': 101.3023903525147, '10681': 101.31139473094711, '7168': 107.9784543821659, '1050': 108.90599210388766, '9686': 109.33921096848204, '993': 115.72306797770739, '5715': 115.80180087568648, '3098': 116.04191228201175, '51422': 116.37994429683737, '5927': 117.93262938776616, '6990': 118.33294034878118, '8914': 119.85389648978708, '4609': 121.38503596288177, '5971': 125.67724298330764, '22889': 125.88444648605356, '1647': 128.10419122820116, '9289': 131.50568114287262, '26036': 131.88953815209797, '6856': 135.38503596288177, '9797': 138.7332513097962, '79073': 144.5299605194383, '10273': 145.53113947309473, '6342': 146.889538152098, '949': 147.7191552653194, '5909': 149.06854956589027, '5111': 155.2757530686362, '994': 155.39639824862704, '9924': 157.32393597034877, '55324': 161.6494267457727, '9133': 163.01018333208881, '7994': 166.95808771798823, '9813': 167.55032718361596, '79643': 170.40658158071585, '80347': 170.48295657138212, '1062': 172.04309123566816, '291': 172.45358552877204, '63874': 173.339210968482, '10450': 173.72933859740823, '2109': 173.84998377739907, '5423': 176.0305499962665, '26020': 177.17901141379227, '4200': 177.54250175883993, '2958': 181.79553025598565, '10904': 182.06619165857742, '6443': 182.68742631539646, '23300': 183.16019955468974, '813': 184.54759342488438, '874': 184.86407982187592, '4312': 185.61105132473023, '1870': 187.29494077915743, '9488': 188.31257368460354, '10972': 190.65805527278638, '9738': 192.94790438589942, '670': 194.44340219668322, '4927': 196.32784868273677, '891': 196.9185333432893, '7319': 197.21229516879035, '11004': 199.3251149240052, '5529': 201.34939430057088, '4846': 202.98590395552318, '26128': 203.285936400725, '10641': 203.41285220041672, '1019': 207.80062192203007, '1026': 210.91617543597647, '10058': 211.0481829017126, '7027': 211.07637499066627, '25966': 212.6784219369641, '10237': 213.50959385526062, '7153': 217.53349738040754, '5777': 218.50176843048465, '5880': 218.64669298704115, '7485': 218.68115569569565, '466': 220.19584121700066, '30001': 221.6392434136839, '11188': 222.7888837848661, '7105': 223.43948948429522, '10730': 223.714063599275, '6194': 226.78416797024036, '27336': 228.59341841928412, '24149': 234.30121139885824, '3337': 235.92753772172173, '51203': 237.94007896112342, '10652': 238.21347412244677, '5641': 238.51077280891704, '1802': 239.41949867153627, '5580': 243.3329403487812, '79090': 244.58596884592689, '8573': 245.4343978182508, '8061': 248.79043858994126, '4638': 249.8499837773991, '3156': 252.85898815583147, '23161': 254.04464604074334, '6709': 256.9263587680653, '1891': 257.3674030574357, '890': 258.02899519119137, '3978': 258.1433697514814, '7538': 260.89580877179884, '642': 261.09518684976877, '5601': 262.3952192949706, '1017': 263.8730842003083, '581': 265.3356741075128, '9375': 267.9745416697779, '3066': 268.64669298704115, '154': 269.18447893125546, '1027': 272.0701043709654, '57761': 273.564047376674, '211': 274.1092828942456, '8050': 274.8499837773991, '9915': 279.224836408192, '142': 280.7230679777074, '1831': 282.42694824489354, '23588': 282.4610351021293, '10513': 284.1437456029001, '9926': 286.6200557031626, '10617': 287.1488372689445, '10285': 287.5260478070503, '5110': 288.02663728387853, '5696': 290.0826456103671, '1500': 290.23619869393724, '1848': 291.22639121326716, '8727': 292.13709913178053, '5588': 294.8691714879203, '93487': 294.9604456253011, '5829': 296.4563192875036, '8985': 297.48022281265054, '3315': 299.1449245565565, '10953': 300.42812719854993, '3775': 302.02819208895363, '5898': 303.04582499439977, '8607': 303.54014385152715, '9712': 303.938900007467, '1398': 304.12964955842324, '5467': 304.93654210015416, '10915': 305.13200746573614, '5743': 308.16019955468977, '25932': 310.0697285195467, '55847': 310.5084149016042, '51116': 312.66352279024954, '23077': 313.9036341965748, '9519': 315.0634578998458, '2920': 316.2111162151339, '10494': 317.3074820185591, '4860': 318.30866097221553, '2037': 319.4484938627277, '2065': 319.81198420777525, '8349': 320.64278027465315, '55604': 320.75243902031747, '54541': 320.92362500933376, '8091': 321.12964955842324, '10013': 322.7066140259177, '5257': 323.74305879046625, '1454': 323.92989562903466, '1213': 325.832350871953, '8878': 325.94163376619855, '1958': 326.89580877179884, '5058': 327.2338407866244, '23271': 328.2514736920705, '11044': 329.5119517625735, '4851': 330.9772754285095, '10525': 331.00273375873155, '23368': 331.39404034131417, '1534': 331.49196094981454, '5708': 331.5667811354056, '5710': 332.0254583302221, '10682': 332.32120221161716, '4282': 333.3329403487812, '29103': 334.5910605119713, '2769': 334.73169650472107, '9703': 335.37876534318093, '25793': 335.78261316516523, '23636': 336.53231842675115, '5290': 337.0188118591025, '392': 338.4966767644402, '6390': 338.5436807124964, '4893': 338.7814342115088, '3895': 339.41794386646114, '9552': 339.6682386048752, '10206': 340.1523741299138, '3162': 340.317665350648, '2523': 344.3086609722156, '58497': 344.9909956215676, '91949': 345.42341138392425, '10782': 345.564047376674, '23659': 345.5706938477936, '7159': 345.6416013209967, '66008': 346.09518684976877, '10797': 347.8499837773991, '701': 348.34548158818285, '6182': 348.5589557106296, '23386': 348.6008679926414, '7398': 348.6161429907747, '2048': 349.1958412170007, '6804': 349.4026688683279, '23224': 349.4488697141463, '9801': 349.47786490533775, '4690': 349.7638013060627, '6117': 350.66823860487517, '3028': 350.88171272732194, '200081': 351.5538640445852, '55556': 352.45476448242846, '1950': 354.33294034878116, '3628': 354.64160132099676, '2184': 355.19037369953753, '11073': 355.5538640445853, '2956': 355.8766210612776, '57215': 355.9353631464977, '836': 356.8080714953873, '22794': 357.44576010399607, '9653': 358.13709913178053, '2597': 360.75361797397386, '998': 362.29611973281385, '8720': 362.50686009652907, '835': 365.82098858620776, '2886': 365.98199124313516, '7106': 367.2048455954331, '10670': 367.992174575224, '9897': 368.2197447421476, '10921': 368.40186576609017, '5048': 369.3760315844494, '6275': 369.98081228947876, '80212': 371.12102103140955, '2064': 373.8464469164298, '3988': 373.8809096250843, '90861': 373.8828916809784, '7077': 373.8969877254553, '84722': 374.16255746200255, '5986': 374.4766859516813, '55825': 375.6584311242051, '873': 377.59186361420905, '3280': 377.6787977883828, '51170': 378.2162078811783, '57178': 378.465323665936, '8851': 378.7990671169549, '3454': 378.95026229321223, '10398': 379.3936644898955, '9261': 379.5973311316721, '63933': 380.747347354273, '29083': 380.861721914563, '9170': 381.9020793914996, '9467': 382.73678817076546, '7048': 383.003912712388, '5289': 383.3646692987041, '23338': 385.19466226334424, '7982': 386.5111486603358, '85236': 386.8523416847119, '9276': 386.89698772545523, '8550': 388.26321182923454, '10898': 388.42303553250554, '7099': 390.1081039405892, '3398': 391.1284706047668, '55008': 392.61142717614894, '11031': 393.7626223524063, '4998': 394.4285030499687, '4331': 394.79043858994123, '11261': 397.2275701669236, '25874': 397.2710372540105, '11344': 397.99726624126845, '7866': 398.79435130232923, '84617': 399.1323833171548, '178': 402.4285030499687, '23585': 402.4355767719072, '10695': 403.82608025225215, '9053': 404.05054080902545, '55012': 404.8209885862077, '6714': 405.73325130979623, '1956': 406.23539559169956}
gene_expression_filter = np.array(gene_expression[list(selected_gene.keys())],dtype="float32")
cell_viability = np.array(cell_viability,dtype="float32")
dtest = xgb.DMatrix(gene_expression_filter,cell_viability)

#Loading model
loaded_model = pickle.load(open("model_adjust\Achilles-L1000-120h_XGBoost_1.dat","rb"))
print("Model Loaded Successfully!")

#Predicting the cell viability
cl_prediction = loaded_model.predict(dtest)
print('Test_Best_Pearson:',sc.stats.pearsonr(cl_prediction,cell_viability))
print("MSE_Test: %.3f" % (mean_squared_error(cl_prediction,cell_viability)))

Model Loaded Successfully!
Test_Best_Pearson: (0.32263407, 8.711526612239323e-29)
MSE_Test: 1.901


In [2]:
#Testing the CTRP-L1000-6h data by using the Achilles-L1000-120h model
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
import pickle
import scipy as sc
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns',15)
warnings.filterwarnings("ignore")

#Retrieving the data
def getData():
    #1.Reading the data
    sig_viability_info = pd.read_table(r"individual_validation\LINCS_CTRP_withoutdose_6h.txt",sep="	",dtype=str)
    sig_viability_info_filter = sig_viability_info[['signature','cell_line','drug','dose','time','cell_viability']]
    sig_GE_info = pd.read_csv(r"individual_validation\LINCS_gene_expression_6h.csv",sep=",",dtype=str)
    sig_GE_info_filter = sig_GE_info[:]

    #2.Combining the two datasets
    sig_viability = pd.merge(sig_viability_info_filter,sig_GE_info_filter,left_on='signature',right_on='cid')
    select_sig_viavility2 = pd.DataFrame(sig_viability).loc[:,'5720':]
    select_sig_viavility_arr = select_sig_viavility2.copy()
    
    #3.Getting the cell viability value
    viability_value = pd.DataFrame(sig_viability).loc[:,'cell_viability']
    
    #4.Return the gene expression matrix and cell viability value
    return select_sig_viavility_arr,viability_value
    
#Retrieving the data
gene_expression,cell_viability = getData()

#Selecting the genes
selected_gene = {'332': 4.208382456402351, '4792': 4.739521929497061, '1846': 4.791617543597649, '983': 9.682334649352057, '5427': 9.890717105754407, '25803': 10.53113947309471, '10221': 10.583235087195298, '3638': 11.42185657884912, '4303': 12.416764912804702, '1514': 12.947904385899411, '6603': 19.421856578849116, '2309': 20.677242983307643, '664': 21.166470174390597, '5357': 21.63533070129589, '3486': 22.567960089062048, '2690': 22.95808771798825, '9217': 24.687426315396475, '6599': 25.958087717988242, '54386': 26.416764912804705, '5236': 29.78652587755323, '7158': 30.588326753239713, '11065': 32.5832350871953, '5050': 35.33294034878119, '3725': 35.374852630792944, '10007': 41.43203991093795, '7157': 43.692517981440886, '16': 44.11437456029001, '2146': 47.10419122820118, '991': 51.661967985174385, '29937': 52.322757016692364, '644': 56.724246931363815, '79071': 57.04700394805617, '4605': 57.578143421150884, '11182': 58.895808771798826, '2058': 63.45867719481645, '6839': 63.73443026345265, '1978': 64.56796008906205, '27244': 70.96317938403267, '1509': 72.29102806676944, '899': 77.20838245640235, '9833': 78.75088421524232, '54505': 79.12573684603525, '1759': 80.16137850834617, '481': 80.30239035251469, '3800': 83.07755394432267, '23': 85.58832675323973, '27032': 87.2021118367015, '3303': 87.8335298256094, '5982': 91.29729868647027, '22908': 91.65060569942914, '10059': 93.65178465308554, '3383': 94.18292412618027, '10180': 94.5832350871953, '11157': 97.25538640445852, '22934': 99.70388026718614, '47': 99.90599210388767, '4216': 101.3023903525147, '10681': 101.31139473094711, '7168': 107.9784543821659, '1050': 108.90599210388766, '9686': 109.33921096848204, '993': 115.72306797770739, '5715': 115.80180087568648, '3098': 116.04191228201175, '51422': 116.37994429683737, '5927': 117.93262938776616, '6990': 118.33294034878118, '8914': 119.85389648978708, '4609': 121.38503596288177, '5971': 125.67724298330764, '22889': 125.88444648605356, '1647': 128.10419122820116, '9289': 131.50568114287262, '26036': 131.88953815209797, '6856': 135.38503596288177, '9797': 138.7332513097962, '79073': 144.5299605194383, '10273': 145.53113947309473, '6342': 146.889538152098, '949': 147.7191552653194, '5909': 149.06854956589027, '5111': 155.2757530686362, '994': 155.39639824862704, '9924': 157.32393597034877, '55324': 161.6494267457727, '9133': 163.01018333208881, '7994': 166.95808771798823, '9813': 167.55032718361596, '79643': 170.40658158071585, '80347': 170.48295657138212, '1062': 172.04309123566816, '291': 172.45358552877204, '63874': 173.339210968482, '10450': 173.72933859740823, '2109': 173.84998377739907, '5423': 176.0305499962665, '26020': 177.17901141379227, '4200': 177.54250175883993, '2958': 181.79553025598565, '10904': 182.06619165857742, '6443': 182.68742631539646, '23300': 183.16019955468974, '813': 184.54759342488438, '874': 184.86407982187592, '4312': 185.61105132473023, '1870': 187.29494077915743, '9488': 188.31257368460354, '10972': 190.65805527278638, '9738': 192.94790438589942, '670': 194.44340219668322, '4927': 196.32784868273677, '891': 196.9185333432893, '7319': 197.21229516879035, '11004': 199.3251149240052, '5529': 201.34939430057088, '4846': 202.98590395552318, '26128': 203.285936400725, '10641': 203.41285220041672, '1019': 207.80062192203007, '1026': 210.91617543597647, '10058': 211.0481829017126, '7027': 211.07637499066627, '25966': 212.6784219369641, '10237': 213.50959385526062, '7153': 217.53349738040754, '5777': 218.50176843048465, '5880': 218.64669298704115, '7485': 218.68115569569565, '466': 220.19584121700066, '30001': 221.6392434136839, '11188': 222.7888837848661, '7105': 223.43948948429522, '10730': 223.714063599275, '6194': 226.78416797024036, '27336': 228.59341841928412, '24149': 234.30121139885824, '3337': 235.92753772172173, '51203': 237.94007896112342, '10652': 238.21347412244677, '5641': 238.51077280891704, '1802': 239.41949867153627, '5580': 243.3329403487812, '79090': 244.58596884592689, '8573': 245.4343978182508, '8061': 248.79043858994126, '4638': 249.8499837773991, '3156': 252.85898815583147, '23161': 254.04464604074334, '6709': 256.9263587680653, '1891': 257.3674030574357, '890': 258.02899519119137, '3978': 258.1433697514814, '7538': 260.89580877179884, '642': 261.09518684976877, '5601': 262.3952192949706, '1017': 263.8730842003083, '581': 265.3356741075128, '9375': 267.9745416697779, '3066': 268.64669298704115, '154': 269.18447893125546, '1027': 272.0701043709654, '57761': 273.564047376674, '211': 274.1092828942456, '8050': 274.8499837773991, '9915': 279.224836408192, '142': 280.7230679777074, '1831': 282.42694824489354, '23588': 282.4610351021293, '10513': 284.1437456029001, '9926': 286.6200557031626, '10617': 287.1488372689445, '10285': 287.5260478070503, '5110': 288.02663728387853, '5696': 290.0826456103671, '1500': 290.23619869393724, '1848': 291.22639121326716, '8727': 292.13709913178053, '5588': 294.8691714879203, '93487': 294.9604456253011, '5829': 296.4563192875036, '8985': 297.48022281265054, '3315': 299.1449245565565, '10953': 300.42812719854993, '3775': 302.02819208895363, '5898': 303.04582499439977, '8607': 303.54014385152715, '9712': 303.938900007467, '1398': 304.12964955842324, '5467': 304.93654210015416, '10915': 305.13200746573614, '5743': 308.16019955468977, '25932': 310.0697285195467, '55847': 310.5084149016042, '51116': 312.66352279024954, '23077': 313.9036341965748, '9519': 315.0634578998458, '2920': 316.2111162151339, '10494': 317.3074820185591, '4860': 318.30866097221553, '2037': 319.4484938627277, '2065': 319.81198420777525, '8349': 320.64278027465315, '55604': 320.75243902031747, '54541': 320.92362500933376, '8091': 321.12964955842324, '10013': 322.7066140259177, '5257': 323.74305879046625, '1454': 323.92989562903466, '1213': 325.832350871953, '8878': 325.94163376619855, '1958': 326.89580877179884, '5058': 327.2338407866244, '23271': 328.2514736920705, '11044': 329.5119517625735, '4851': 330.9772754285095, '10525': 331.00273375873155, '23368': 331.39404034131417, '1534': 331.49196094981454, '5708': 331.5667811354056, '5710': 332.0254583302221, '10682': 332.32120221161716, '4282': 333.3329403487812, '29103': 334.5910605119713, '2769': 334.73169650472107, '9703': 335.37876534318093, '25793': 335.78261316516523, '23636': 336.53231842675115, '5290': 337.0188118591025, '392': 338.4966767644402, '6390': 338.5436807124964, '4893': 338.7814342115088, '3895': 339.41794386646114, '9552': 339.6682386048752, '10206': 340.1523741299138, '3162': 340.317665350648, '2523': 344.3086609722156, '58497': 344.9909956215676, '91949': 345.42341138392425, '10782': 345.564047376674, '23659': 345.5706938477936, '7159': 345.6416013209967, '66008': 346.09518684976877, '10797': 347.8499837773991, '701': 348.34548158818285, '6182': 348.5589557106296, '23386': 348.6008679926414, '7398': 348.6161429907747, '2048': 349.1958412170007, '6804': 349.4026688683279, '23224': 349.4488697141463, '9801': 349.47786490533775, '4690': 349.7638013060627, '6117': 350.66823860487517, '3028': 350.88171272732194, '200081': 351.5538640445852, '55556': 352.45476448242846, '1950': 354.33294034878116, '3628': 354.64160132099676, '2184': 355.19037369953753, '11073': 355.5538640445853, '2956': 355.8766210612776, '57215': 355.9353631464977, '836': 356.8080714953873, '22794': 357.44576010399607, '9653': 358.13709913178053, '2597': 360.75361797397386, '998': 362.29611973281385, '8720': 362.50686009652907, '835': 365.82098858620776, '2886': 365.98199124313516, '7106': 367.2048455954331, '10670': 367.992174575224, '9897': 368.2197447421476, '10921': 368.40186576609017, '5048': 369.3760315844494, '6275': 369.98081228947876, '80212': 371.12102103140955, '2064': 373.8464469164298, '3988': 373.8809096250843, '90861': 373.8828916809784, '7077': 373.8969877254553, '84722': 374.16255746200255, '5986': 374.4766859516813, '55825': 375.6584311242051, '873': 377.59186361420905, '3280': 377.6787977883828, '51170': 378.2162078811783, '57178': 378.465323665936, '8851': 378.7990671169549, '3454': 378.95026229321223, '10398': 379.3936644898955, '9261': 379.5973311316721, '63933': 380.747347354273, '29083': 380.861721914563, '9170': 381.9020793914996, '9467': 382.73678817076546, '7048': 383.003912712388, '5289': 383.3646692987041, '23338': 385.19466226334424, '7982': 386.5111486603358, '85236': 386.8523416847119, '9276': 386.89698772545523, '8550': 388.26321182923454, '10898': 388.42303553250554, '7099': 390.1081039405892, '3398': 391.1284706047668, '55008': 392.61142717614894, '11031': 393.7626223524063, '4998': 394.4285030499687, '4331': 394.79043858994123, '11261': 397.2275701669236, '25874': 397.2710372540105, '11344': 397.99726624126845, '7866': 398.79435130232923, '84617': 399.1323833171548, '178': 402.4285030499687, '23585': 402.4355767719072, '10695': 403.82608025225215, '9053': 404.05054080902545, '55012': 404.8209885862077, '6714': 405.73325130979623, '1956': 406.23539559169956}
gene_expression_filter = np.array(gene_expression[list(selected_gene.keys())],dtype="float32")
cell_viability = np.array(cell_viability,dtype="float32")
dtest = xgb.DMatrix(gene_expression_filter,cell_viability)

#Loading model
loaded_model = pickle.load(open("model_adjust\Achilles-L1000-120h_XGBoost_1.dat","rb"))
print("Model Loaded Successfully!")

#Predicting the cell viability
cl_prediction = loaded_model.predict(dtest)
print('Test_Best_Pearson:',sc.stats.pearsonr(cl_prediction,cell_viability))
print("MSE_Test: %.3f" % (mean_squared_error(cl_prediction,cell_viability)))

Model Loaded Successfully!
Test_Best_Pearson: (0.3574129, 2.4439719004420354e-272)
MSE_Test: 2.597


In [3]:
#Testing the CTRP-L1000-24h data by using the Achilles-L1000-120h model
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
import pickle
import scipy as sc
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns',15)
warnings.filterwarnings("ignore")

#Retrieving the data
def getData():
    #1.Reading the data
    sig_viability_info = pd.read_table(r"individual_validation\LINCS_CTRP_withoutdose_24h.txt",sep="	",dtype=str)
    sig_viability_info_filter = sig_viability_info[['signature','cell_line','drug','dose','time','cell_viability']]
    sig_GE_info = pd.read_csv(r"individual_validation\LINCS_gene_expression_24h.csv",sep=",",dtype=str)
    sig_GE_info_filter = sig_GE_info[:]

    #2.Combining the two datasets
    sig_viability = pd.merge(sig_viability_info_filter,sig_GE_info_filter,left_on='signature',right_on='cid')
    select_sig_viavility2 = pd.DataFrame(sig_viability).loc[:,'5720':]
    select_sig_viavility_arr = select_sig_viavility2.copy()
    
    #3.Getting the cell viability value
    viability_value = pd.DataFrame(sig_viability).loc[:,'cell_viability']
    
    #4.Return the gene expression matrix and cell viability value
    return select_sig_viavility_arr,viability_value
    
#Retrieving the data
gene_expression,cell_viability = getData()

#Selecting the genes
selected_gene = {'332': 4.208382456402351, '4792': 4.739521929497061, '1846': 4.791617543597649, '983': 9.682334649352057, '5427': 9.890717105754407, '25803': 10.53113947309471, '10221': 10.583235087195298, '3638': 11.42185657884912, '4303': 12.416764912804702, '1514': 12.947904385899411, '6603': 19.421856578849116, '2309': 20.677242983307643, '664': 21.166470174390597, '5357': 21.63533070129589, '3486': 22.567960089062048, '2690': 22.95808771798825, '9217': 24.687426315396475, '6599': 25.958087717988242, '54386': 26.416764912804705, '5236': 29.78652587755323, '7158': 30.588326753239713, '11065': 32.5832350871953, '5050': 35.33294034878119, '3725': 35.374852630792944, '10007': 41.43203991093795, '7157': 43.692517981440886, '16': 44.11437456029001, '2146': 47.10419122820118, '991': 51.661967985174385, '29937': 52.322757016692364, '644': 56.724246931363815, '79071': 57.04700394805617, '4605': 57.578143421150884, '11182': 58.895808771798826, '2058': 63.45867719481645, '6839': 63.73443026345265, '1978': 64.56796008906205, '27244': 70.96317938403267, '1509': 72.29102806676944, '899': 77.20838245640235, '9833': 78.75088421524232, '54505': 79.12573684603525, '1759': 80.16137850834617, '481': 80.30239035251469, '3800': 83.07755394432267, '23': 85.58832675323973, '27032': 87.2021118367015, '3303': 87.8335298256094, '5982': 91.29729868647027, '22908': 91.65060569942914, '10059': 93.65178465308554, '3383': 94.18292412618027, '10180': 94.5832350871953, '11157': 97.25538640445852, '22934': 99.70388026718614, '47': 99.90599210388767, '4216': 101.3023903525147, '10681': 101.31139473094711, '7168': 107.9784543821659, '1050': 108.90599210388766, '9686': 109.33921096848204, '993': 115.72306797770739, '5715': 115.80180087568648, '3098': 116.04191228201175, '51422': 116.37994429683737, '5927': 117.93262938776616, '6990': 118.33294034878118, '8914': 119.85389648978708, '4609': 121.38503596288177, '5971': 125.67724298330764, '22889': 125.88444648605356, '1647': 128.10419122820116, '9289': 131.50568114287262, '26036': 131.88953815209797, '6856': 135.38503596288177, '9797': 138.7332513097962, '79073': 144.5299605194383, '10273': 145.53113947309473, '6342': 146.889538152098, '949': 147.7191552653194, '5909': 149.06854956589027, '5111': 155.2757530686362, '994': 155.39639824862704, '9924': 157.32393597034877, '55324': 161.6494267457727, '9133': 163.01018333208881, '7994': 166.95808771798823, '9813': 167.55032718361596, '79643': 170.40658158071585, '80347': 170.48295657138212, '1062': 172.04309123566816, '291': 172.45358552877204, '63874': 173.339210968482, '10450': 173.72933859740823, '2109': 173.84998377739907, '5423': 176.0305499962665, '26020': 177.17901141379227, '4200': 177.54250175883993, '2958': 181.79553025598565, '10904': 182.06619165857742, '6443': 182.68742631539646, '23300': 183.16019955468974, '813': 184.54759342488438, '874': 184.86407982187592, '4312': 185.61105132473023, '1870': 187.29494077915743, '9488': 188.31257368460354, '10972': 190.65805527278638, '9738': 192.94790438589942, '670': 194.44340219668322, '4927': 196.32784868273677, '891': 196.9185333432893, '7319': 197.21229516879035, '11004': 199.3251149240052, '5529': 201.34939430057088, '4846': 202.98590395552318, '26128': 203.285936400725, '10641': 203.41285220041672, '1019': 207.80062192203007, '1026': 210.91617543597647, '10058': 211.0481829017126, '7027': 211.07637499066627, '25966': 212.6784219369641, '10237': 213.50959385526062, '7153': 217.53349738040754, '5777': 218.50176843048465, '5880': 218.64669298704115, '7485': 218.68115569569565, '466': 220.19584121700066, '30001': 221.6392434136839, '11188': 222.7888837848661, '7105': 223.43948948429522, '10730': 223.714063599275, '6194': 226.78416797024036, '27336': 228.59341841928412, '24149': 234.30121139885824, '3337': 235.92753772172173, '51203': 237.94007896112342, '10652': 238.21347412244677, '5641': 238.51077280891704, '1802': 239.41949867153627, '5580': 243.3329403487812, '79090': 244.58596884592689, '8573': 245.4343978182508, '8061': 248.79043858994126, '4638': 249.8499837773991, '3156': 252.85898815583147, '23161': 254.04464604074334, '6709': 256.9263587680653, '1891': 257.3674030574357, '890': 258.02899519119137, '3978': 258.1433697514814, '7538': 260.89580877179884, '642': 261.09518684976877, '5601': 262.3952192949706, '1017': 263.8730842003083, '581': 265.3356741075128, '9375': 267.9745416697779, '3066': 268.64669298704115, '154': 269.18447893125546, '1027': 272.0701043709654, '57761': 273.564047376674, '211': 274.1092828942456, '8050': 274.8499837773991, '9915': 279.224836408192, '142': 280.7230679777074, '1831': 282.42694824489354, '23588': 282.4610351021293, '10513': 284.1437456029001, '9926': 286.6200557031626, '10617': 287.1488372689445, '10285': 287.5260478070503, '5110': 288.02663728387853, '5696': 290.0826456103671, '1500': 290.23619869393724, '1848': 291.22639121326716, '8727': 292.13709913178053, '5588': 294.8691714879203, '93487': 294.9604456253011, '5829': 296.4563192875036, '8985': 297.48022281265054, '3315': 299.1449245565565, '10953': 300.42812719854993, '3775': 302.02819208895363, '5898': 303.04582499439977, '8607': 303.54014385152715, '9712': 303.938900007467, '1398': 304.12964955842324, '5467': 304.93654210015416, '10915': 305.13200746573614, '5743': 308.16019955468977, '25932': 310.0697285195467, '55847': 310.5084149016042, '51116': 312.66352279024954, '23077': 313.9036341965748, '9519': 315.0634578998458, '2920': 316.2111162151339, '10494': 317.3074820185591, '4860': 318.30866097221553, '2037': 319.4484938627277, '2065': 319.81198420777525, '8349': 320.64278027465315, '55604': 320.75243902031747, '54541': 320.92362500933376, '8091': 321.12964955842324, '10013': 322.7066140259177, '5257': 323.74305879046625, '1454': 323.92989562903466, '1213': 325.832350871953, '8878': 325.94163376619855, '1958': 326.89580877179884, '5058': 327.2338407866244, '23271': 328.2514736920705, '11044': 329.5119517625735, '4851': 330.9772754285095, '10525': 331.00273375873155, '23368': 331.39404034131417, '1534': 331.49196094981454, '5708': 331.5667811354056, '5710': 332.0254583302221, '10682': 332.32120221161716, '4282': 333.3329403487812, '29103': 334.5910605119713, '2769': 334.73169650472107, '9703': 335.37876534318093, '25793': 335.78261316516523, '23636': 336.53231842675115, '5290': 337.0188118591025, '392': 338.4966767644402, '6390': 338.5436807124964, '4893': 338.7814342115088, '3895': 339.41794386646114, '9552': 339.6682386048752, '10206': 340.1523741299138, '3162': 340.317665350648, '2523': 344.3086609722156, '58497': 344.9909956215676, '91949': 345.42341138392425, '10782': 345.564047376674, '23659': 345.5706938477936, '7159': 345.6416013209967, '66008': 346.09518684976877, '10797': 347.8499837773991, '701': 348.34548158818285, '6182': 348.5589557106296, '23386': 348.6008679926414, '7398': 348.6161429907747, '2048': 349.1958412170007, '6804': 349.4026688683279, '23224': 349.4488697141463, '9801': 349.47786490533775, '4690': 349.7638013060627, '6117': 350.66823860487517, '3028': 350.88171272732194, '200081': 351.5538640445852, '55556': 352.45476448242846, '1950': 354.33294034878116, '3628': 354.64160132099676, '2184': 355.19037369953753, '11073': 355.5538640445853, '2956': 355.8766210612776, '57215': 355.9353631464977, '836': 356.8080714953873, '22794': 357.44576010399607, '9653': 358.13709913178053, '2597': 360.75361797397386, '998': 362.29611973281385, '8720': 362.50686009652907, '835': 365.82098858620776, '2886': 365.98199124313516, '7106': 367.2048455954331, '10670': 367.992174575224, '9897': 368.2197447421476, '10921': 368.40186576609017, '5048': 369.3760315844494, '6275': 369.98081228947876, '80212': 371.12102103140955, '2064': 373.8464469164298, '3988': 373.8809096250843, '90861': 373.8828916809784, '7077': 373.8969877254553, '84722': 374.16255746200255, '5986': 374.4766859516813, '55825': 375.6584311242051, '873': 377.59186361420905, '3280': 377.6787977883828, '51170': 378.2162078811783, '57178': 378.465323665936, '8851': 378.7990671169549, '3454': 378.95026229321223, '10398': 379.3936644898955, '9261': 379.5973311316721, '63933': 380.747347354273, '29083': 380.861721914563, '9170': 381.9020793914996, '9467': 382.73678817076546, '7048': 383.003912712388, '5289': 383.3646692987041, '23338': 385.19466226334424, '7982': 386.5111486603358, '85236': 386.8523416847119, '9276': 386.89698772545523, '8550': 388.26321182923454, '10898': 388.42303553250554, '7099': 390.1081039405892, '3398': 391.1284706047668, '55008': 392.61142717614894, '11031': 393.7626223524063, '4998': 394.4285030499687, '4331': 394.79043858994123, '11261': 397.2275701669236, '25874': 397.2710372540105, '11344': 397.99726624126845, '7866': 398.79435130232923, '84617': 399.1323833171548, '178': 402.4285030499687, '23585': 402.4355767719072, '10695': 403.82608025225215, '9053': 404.05054080902545, '55012': 404.8209885862077, '6714': 405.73325130979623, '1956': 406.23539559169956}
gene_expression_filter = np.array(gene_expression[list(selected_gene.keys())],dtype="float32")
cell_viability = np.array(cell_viability,dtype="float32")
dtest = xgb.DMatrix(gene_expression_filter,cell_viability)

#Loading model
loaded_model = pickle.load(open("model_adjust\Achilles-L1000-120h_XGBoost_1.dat","rb"))
print("Model Loaded Successfully!")

#Predicting the cell viability
cl_prediction = loaded_model.predict(dtest)
print('Test_Best_Pearson:',sc.stats.pearsonr(cl_prediction,cell_viability))
print("MSE_Test: %.3f" % (mean_squared_error(cl_prediction,cell_viability)))

Model Loaded Successfully!
Test_Best_Pearson: (0.6950701, 0.0)
MSE_Test: 4.123


In [5]:
#Testing the Achilles-L1000-96h data by using the Achilles-L1000-120h model
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
import pickle
import scipy as sc
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns',15)
warnings.filterwarnings("ignore")

#Retrieving the data
def getData():
    #1.Reading the data
    sig_viability_info = pd.read_table(r"generate\Achilles_L1000_PhaseI_96h_unique.txt",sep="	",dtype=str)
    sig_viability_info_filter = sig_viability_info[['signature','cell_line','drug','pert_type','time','cell_viability']]
    sig_GE_info = pd.read_csv(r"generate\Achilles_L1000_PhaseI_gene_expression_96h.csv",sep=",",dtype=str)
    sig_GE_info_filter = sig_GE_info[:]

    #2.Combining the two datasets
    sig_viability = pd.merge(sig_viability_info_filter,sig_GE_info_filter,left_on='signature',right_on='cid')
    select_sig_viavility2 = pd.DataFrame(sig_viability).loc[:,'5720':]
    select_sig_viavility_arr = select_sig_viavility2.copy()
    
    #3.Getting the cell viability value
    viability_value = pd.DataFrame(sig_viability).loc[:,'cell_viability']
    
    #4.Return the gene expression matrix and cell viability value
    return select_sig_viavility_arr,viability_value
    
#Retrieving the data
gene_expression,cell_viability = getData()

#Selecting the genes
selected_gene = {'332': 4.208382456402351, '4792': 4.739521929497061, '1846': 4.791617543597649, '983': 9.682334649352057, '5427': 9.890717105754407, '25803': 10.53113947309471, '10221': 10.583235087195298, '3638': 11.42185657884912, '4303': 12.416764912804702, '1514': 12.947904385899411, '6603': 19.421856578849116, '2309': 20.677242983307643, '664': 21.166470174390597, '5357': 21.63533070129589, '3486': 22.567960089062048, '2690': 22.95808771798825, '9217': 24.687426315396475, '6599': 25.958087717988242, '54386': 26.416764912804705, '5236': 29.78652587755323, '7158': 30.588326753239713, '11065': 32.5832350871953, '5050': 35.33294034878119, '3725': 35.374852630792944, '10007': 41.43203991093795, '7157': 43.692517981440886, '16': 44.11437456029001, '2146': 47.10419122820118, '991': 51.661967985174385, '29937': 52.322757016692364, '644': 56.724246931363815, '79071': 57.04700394805617, '4605': 57.578143421150884, '11182': 58.895808771798826, '2058': 63.45867719481645, '6839': 63.73443026345265, '1978': 64.56796008906205, '27244': 70.96317938403267, '1509': 72.29102806676944, '899': 77.20838245640235, '9833': 78.75088421524232, '54505': 79.12573684603525, '1759': 80.16137850834617, '481': 80.30239035251469, '3800': 83.07755394432267, '23': 85.58832675323973, '27032': 87.2021118367015, '3303': 87.8335298256094, '5982': 91.29729868647027, '22908': 91.65060569942914, '10059': 93.65178465308554, '3383': 94.18292412618027, '10180': 94.5832350871953, '11157': 97.25538640445852, '22934': 99.70388026718614, '47': 99.90599210388767, '4216': 101.3023903525147, '10681': 101.31139473094711, '7168': 107.9784543821659, '1050': 108.90599210388766, '9686': 109.33921096848204, '993': 115.72306797770739, '5715': 115.80180087568648, '3098': 116.04191228201175, '51422': 116.37994429683737, '5927': 117.93262938776616, '6990': 118.33294034878118, '8914': 119.85389648978708, '4609': 121.38503596288177, '5971': 125.67724298330764, '22889': 125.88444648605356, '1647': 128.10419122820116, '9289': 131.50568114287262, '26036': 131.88953815209797, '6856': 135.38503596288177, '9797': 138.7332513097962, '79073': 144.5299605194383, '10273': 145.53113947309473, '6342': 146.889538152098, '949': 147.7191552653194, '5909': 149.06854956589027, '5111': 155.2757530686362, '994': 155.39639824862704, '9924': 157.32393597034877, '55324': 161.6494267457727, '9133': 163.01018333208881, '7994': 166.95808771798823, '9813': 167.55032718361596, '79643': 170.40658158071585, '80347': 170.48295657138212, '1062': 172.04309123566816, '291': 172.45358552877204, '63874': 173.339210968482, '10450': 173.72933859740823, '2109': 173.84998377739907, '5423': 176.0305499962665, '26020': 177.17901141379227, '4200': 177.54250175883993, '2958': 181.79553025598565, '10904': 182.06619165857742, '6443': 182.68742631539646, '23300': 183.16019955468974, '813': 184.54759342488438, '874': 184.86407982187592, '4312': 185.61105132473023, '1870': 187.29494077915743, '9488': 188.31257368460354, '10972': 190.65805527278638, '9738': 192.94790438589942, '670': 194.44340219668322, '4927': 196.32784868273677, '891': 196.9185333432893, '7319': 197.21229516879035, '11004': 199.3251149240052, '5529': 201.34939430057088, '4846': 202.98590395552318, '26128': 203.285936400725, '10641': 203.41285220041672, '1019': 207.80062192203007, '1026': 210.91617543597647, '10058': 211.0481829017126, '7027': 211.07637499066627, '25966': 212.6784219369641, '10237': 213.50959385526062, '7153': 217.53349738040754, '5777': 218.50176843048465, '5880': 218.64669298704115, '7485': 218.68115569569565, '466': 220.19584121700066, '30001': 221.6392434136839, '11188': 222.7888837848661, '7105': 223.43948948429522, '10730': 223.714063599275, '6194': 226.78416797024036, '27336': 228.59341841928412, '24149': 234.30121139885824, '3337': 235.92753772172173, '51203': 237.94007896112342, '10652': 238.21347412244677, '5641': 238.51077280891704, '1802': 239.41949867153627, '5580': 243.3329403487812, '79090': 244.58596884592689, '8573': 245.4343978182508, '8061': 248.79043858994126, '4638': 249.8499837773991, '3156': 252.85898815583147, '23161': 254.04464604074334, '6709': 256.9263587680653, '1891': 257.3674030574357, '890': 258.02899519119137, '3978': 258.1433697514814, '7538': 260.89580877179884, '642': 261.09518684976877, '5601': 262.3952192949706, '1017': 263.8730842003083, '581': 265.3356741075128, '9375': 267.9745416697779, '3066': 268.64669298704115, '154': 269.18447893125546, '1027': 272.0701043709654, '57761': 273.564047376674, '211': 274.1092828942456, '8050': 274.8499837773991, '9915': 279.224836408192, '142': 280.7230679777074, '1831': 282.42694824489354, '23588': 282.4610351021293, '10513': 284.1437456029001, '9926': 286.6200557031626, '10617': 287.1488372689445, '10285': 287.5260478070503, '5110': 288.02663728387853, '5696': 290.0826456103671, '1500': 290.23619869393724, '1848': 291.22639121326716, '8727': 292.13709913178053, '5588': 294.8691714879203, '93487': 294.9604456253011, '5829': 296.4563192875036, '8985': 297.48022281265054, '3315': 299.1449245565565, '10953': 300.42812719854993, '3775': 302.02819208895363, '5898': 303.04582499439977, '8607': 303.54014385152715, '9712': 303.938900007467, '1398': 304.12964955842324, '5467': 304.93654210015416, '10915': 305.13200746573614, '5743': 308.16019955468977, '25932': 310.0697285195467, '55847': 310.5084149016042, '51116': 312.66352279024954, '23077': 313.9036341965748, '9519': 315.0634578998458, '2920': 316.2111162151339, '10494': 317.3074820185591, '4860': 318.30866097221553, '2037': 319.4484938627277, '2065': 319.81198420777525, '8349': 320.64278027465315, '55604': 320.75243902031747, '54541': 320.92362500933376, '8091': 321.12964955842324, '10013': 322.7066140259177, '5257': 323.74305879046625, '1454': 323.92989562903466, '1213': 325.832350871953, '8878': 325.94163376619855, '1958': 326.89580877179884, '5058': 327.2338407866244, '23271': 328.2514736920705, '11044': 329.5119517625735, '4851': 330.9772754285095, '10525': 331.00273375873155, '23368': 331.39404034131417, '1534': 331.49196094981454, '5708': 331.5667811354056, '5710': 332.0254583302221, '10682': 332.32120221161716, '4282': 333.3329403487812, '29103': 334.5910605119713, '2769': 334.73169650472107, '9703': 335.37876534318093, '25793': 335.78261316516523, '23636': 336.53231842675115, '5290': 337.0188118591025, '392': 338.4966767644402, '6390': 338.5436807124964, '4893': 338.7814342115088, '3895': 339.41794386646114, '9552': 339.6682386048752, '10206': 340.1523741299138, '3162': 340.317665350648, '2523': 344.3086609722156, '58497': 344.9909956215676, '91949': 345.42341138392425, '10782': 345.564047376674, '23659': 345.5706938477936, '7159': 345.6416013209967, '66008': 346.09518684976877, '10797': 347.8499837773991, '701': 348.34548158818285, '6182': 348.5589557106296, '23386': 348.6008679926414, '7398': 348.6161429907747, '2048': 349.1958412170007, '6804': 349.4026688683279, '23224': 349.4488697141463, '9801': 349.47786490533775, '4690': 349.7638013060627, '6117': 350.66823860487517, '3028': 350.88171272732194, '200081': 351.5538640445852, '55556': 352.45476448242846, '1950': 354.33294034878116, '3628': 354.64160132099676, '2184': 355.19037369953753, '11073': 355.5538640445853, '2956': 355.8766210612776, '57215': 355.9353631464977, '836': 356.8080714953873, '22794': 357.44576010399607, '9653': 358.13709913178053, '2597': 360.75361797397386, '998': 362.29611973281385, '8720': 362.50686009652907, '835': 365.82098858620776, '2886': 365.98199124313516, '7106': 367.2048455954331, '10670': 367.992174575224, '9897': 368.2197447421476, '10921': 368.40186576609017, '5048': 369.3760315844494, '6275': 369.98081228947876, '80212': 371.12102103140955, '2064': 373.8464469164298, '3988': 373.8809096250843, '90861': 373.8828916809784, '7077': 373.8969877254553, '84722': 374.16255746200255, '5986': 374.4766859516813, '55825': 375.6584311242051, '873': 377.59186361420905, '3280': 377.6787977883828, '51170': 378.2162078811783, '57178': 378.465323665936, '8851': 378.7990671169549, '3454': 378.95026229321223, '10398': 379.3936644898955, '9261': 379.5973311316721, '63933': 380.747347354273, '29083': 380.861721914563, '9170': 381.9020793914996, '9467': 382.73678817076546, '7048': 383.003912712388, '5289': 383.3646692987041, '23338': 385.19466226334424, '7982': 386.5111486603358, '85236': 386.8523416847119, '9276': 386.89698772545523, '8550': 388.26321182923454, '10898': 388.42303553250554, '7099': 390.1081039405892, '3398': 391.1284706047668, '55008': 392.61142717614894, '11031': 393.7626223524063, '4998': 394.4285030499687, '4331': 394.79043858994123, '11261': 397.2275701669236, '25874': 397.2710372540105, '11344': 397.99726624126845, '7866': 398.79435130232923, '84617': 399.1323833171548, '178': 402.4285030499687, '23585': 402.4355767719072, '10695': 403.82608025225215, '9053': 404.05054080902545, '55012': 404.8209885862077, '6714': 405.73325130979623, '1956': 406.23539559169956}
gene_expression_filter = np.array(gene_expression[list(selected_gene.keys())],dtype="float32")
cell_viability = np.array(cell_viability,dtype="float32")
dtest = xgb.DMatrix(gene_expression_filter,cell_viability)

#Loading model
loaded_model = pickle.load(open("model_adjust\Achilles-L1000-120h_XGBoost_1.dat","rb"))
print("Model Loaded Successfully!")

#Predicting the cell viability
cl_prediction = loaded_model.predict(dtest)
print('Test_Best_Pearson:',sc.stats.pearsonr(cl_prediction,cell_viability))
print("MSE_Test: %.3f" % (mean_squared_error(cl_prediction,cell_viability)))

Model Loaded Successfully!
Test_Best_Pearson: (0.4337745, 0.0)
MSE_Test: 1.862


In [6]:
#Testing the Achilles-L1000-120h data by using the Achilles-L1000-120h model
import pandas as pd
import numpy as np
import warnings
import xgboost as xgb
import pickle
import scipy as sc
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns',15)
warnings.filterwarnings("ignore")

#Retrieving the data
def getData():
    #1.Reading the data
    sig_viability_info = pd.read_table(r"generate\Achilles_L1000_PhaseI_144h_unique.txt",sep="	",dtype=str)
    sig_viability_info_filter = sig_viability_info[['signature','cell_line','drug','pert_type','time','cell_viability']]
    sig_GE_info = pd.read_csv(r"generate\Achilles_L1000_PhaseI_gene_expression_144h.csv",sep=",",dtype=str)
    sig_GE_info_filter = sig_GE_info[:]

    #2.Combining the two datasets
    sig_viability = pd.merge(sig_viability_info_filter,sig_GE_info_filter,left_on='signature',right_on='cid')
    select_sig_viavility2 = pd.DataFrame(sig_viability).loc[:,'5720':]
    select_sig_viavility_arr = select_sig_viavility2.copy()
    
    #3.Getting the cell viability value
    viability_value = pd.DataFrame(sig_viability).loc[:,'cell_viability']
    
    #4.Return the gene expression matrix and cell viability value
    return select_sig_viavility_arr,viability_value
    
#Retrieving the data
gene_expression,cell_viability = getData()

#Selecting the genes
selected_gene = {'332': 4.208382456402351, '4792': 4.739521929497061, '1846': 4.791617543597649, '983': 9.682334649352057, '5427': 9.890717105754407, '25803': 10.53113947309471, '10221': 10.583235087195298, '3638': 11.42185657884912, '4303': 12.416764912804702, '1514': 12.947904385899411, '6603': 19.421856578849116, '2309': 20.677242983307643, '664': 21.166470174390597, '5357': 21.63533070129589, '3486': 22.567960089062048, '2690': 22.95808771798825, '9217': 24.687426315396475, '6599': 25.958087717988242, '54386': 26.416764912804705, '5236': 29.78652587755323, '7158': 30.588326753239713, '11065': 32.5832350871953, '5050': 35.33294034878119, '3725': 35.374852630792944, '10007': 41.43203991093795, '7157': 43.692517981440886, '16': 44.11437456029001, '2146': 47.10419122820118, '991': 51.661967985174385, '29937': 52.322757016692364, '644': 56.724246931363815, '79071': 57.04700394805617, '4605': 57.578143421150884, '11182': 58.895808771798826, '2058': 63.45867719481645, '6839': 63.73443026345265, '1978': 64.56796008906205, '27244': 70.96317938403267, '1509': 72.29102806676944, '899': 77.20838245640235, '9833': 78.75088421524232, '54505': 79.12573684603525, '1759': 80.16137850834617, '481': 80.30239035251469, '3800': 83.07755394432267, '23': 85.58832675323973, '27032': 87.2021118367015, '3303': 87.8335298256094, '5982': 91.29729868647027, '22908': 91.65060569942914, '10059': 93.65178465308554, '3383': 94.18292412618027, '10180': 94.5832350871953, '11157': 97.25538640445852, '22934': 99.70388026718614, '47': 99.90599210388767, '4216': 101.3023903525147, '10681': 101.31139473094711, '7168': 107.9784543821659, '1050': 108.90599210388766, '9686': 109.33921096848204, '993': 115.72306797770739, '5715': 115.80180087568648, '3098': 116.04191228201175, '51422': 116.37994429683737, '5927': 117.93262938776616, '6990': 118.33294034878118, '8914': 119.85389648978708, '4609': 121.38503596288177, '5971': 125.67724298330764, '22889': 125.88444648605356, '1647': 128.10419122820116, '9289': 131.50568114287262, '26036': 131.88953815209797, '6856': 135.38503596288177, '9797': 138.7332513097962, '79073': 144.5299605194383, '10273': 145.53113947309473, '6342': 146.889538152098, '949': 147.7191552653194, '5909': 149.06854956589027, '5111': 155.2757530686362, '994': 155.39639824862704, '9924': 157.32393597034877, '55324': 161.6494267457727, '9133': 163.01018333208881, '7994': 166.95808771798823, '9813': 167.55032718361596, '79643': 170.40658158071585, '80347': 170.48295657138212, '1062': 172.04309123566816, '291': 172.45358552877204, '63874': 173.339210968482, '10450': 173.72933859740823, '2109': 173.84998377739907, '5423': 176.0305499962665, '26020': 177.17901141379227, '4200': 177.54250175883993, '2958': 181.79553025598565, '10904': 182.06619165857742, '6443': 182.68742631539646, '23300': 183.16019955468974, '813': 184.54759342488438, '874': 184.86407982187592, '4312': 185.61105132473023, '1870': 187.29494077915743, '9488': 188.31257368460354, '10972': 190.65805527278638, '9738': 192.94790438589942, '670': 194.44340219668322, '4927': 196.32784868273677, '891': 196.9185333432893, '7319': 197.21229516879035, '11004': 199.3251149240052, '5529': 201.34939430057088, '4846': 202.98590395552318, '26128': 203.285936400725, '10641': 203.41285220041672, '1019': 207.80062192203007, '1026': 210.91617543597647, '10058': 211.0481829017126, '7027': 211.07637499066627, '25966': 212.6784219369641, '10237': 213.50959385526062, '7153': 217.53349738040754, '5777': 218.50176843048465, '5880': 218.64669298704115, '7485': 218.68115569569565, '466': 220.19584121700066, '30001': 221.6392434136839, '11188': 222.7888837848661, '7105': 223.43948948429522, '10730': 223.714063599275, '6194': 226.78416797024036, '27336': 228.59341841928412, '24149': 234.30121139885824, '3337': 235.92753772172173, '51203': 237.94007896112342, '10652': 238.21347412244677, '5641': 238.51077280891704, '1802': 239.41949867153627, '5580': 243.3329403487812, '79090': 244.58596884592689, '8573': 245.4343978182508, '8061': 248.79043858994126, '4638': 249.8499837773991, '3156': 252.85898815583147, '23161': 254.04464604074334, '6709': 256.9263587680653, '1891': 257.3674030574357, '890': 258.02899519119137, '3978': 258.1433697514814, '7538': 260.89580877179884, '642': 261.09518684976877, '5601': 262.3952192949706, '1017': 263.8730842003083, '581': 265.3356741075128, '9375': 267.9745416697779, '3066': 268.64669298704115, '154': 269.18447893125546, '1027': 272.0701043709654, '57761': 273.564047376674, '211': 274.1092828942456, '8050': 274.8499837773991, '9915': 279.224836408192, '142': 280.7230679777074, '1831': 282.42694824489354, '23588': 282.4610351021293, '10513': 284.1437456029001, '9926': 286.6200557031626, '10617': 287.1488372689445, '10285': 287.5260478070503, '5110': 288.02663728387853, '5696': 290.0826456103671, '1500': 290.23619869393724, '1848': 291.22639121326716, '8727': 292.13709913178053, '5588': 294.8691714879203, '93487': 294.9604456253011, '5829': 296.4563192875036, '8985': 297.48022281265054, '3315': 299.1449245565565, '10953': 300.42812719854993, '3775': 302.02819208895363, '5898': 303.04582499439977, '8607': 303.54014385152715, '9712': 303.938900007467, '1398': 304.12964955842324, '5467': 304.93654210015416, '10915': 305.13200746573614, '5743': 308.16019955468977, '25932': 310.0697285195467, '55847': 310.5084149016042, '51116': 312.66352279024954, '23077': 313.9036341965748, '9519': 315.0634578998458, '2920': 316.2111162151339, '10494': 317.3074820185591, '4860': 318.30866097221553, '2037': 319.4484938627277, '2065': 319.81198420777525, '8349': 320.64278027465315, '55604': 320.75243902031747, '54541': 320.92362500933376, '8091': 321.12964955842324, '10013': 322.7066140259177, '5257': 323.74305879046625, '1454': 323.92989562903466, '1213': 325.832350871953, '8878': 325.94163376619855, '1958': 326.89580877179884, '5058': 327.2338407866244, '23271': 328.2514736920705, '11044': 329.5119517625735, '4851': 330.9772754285095, '10525': 331.00273375873155, '23368': 331.39404034131417, '1534': 331.49196094981454, '5708': 331.5667811354056, '5710': 332.0254583302221, '10682': 332.32120221161716, '4282': 333.3329403487812, '29103': 334.5910605119713, '2769': 334.73169650472107, '9703': 335.37876534318093, '25793': 335.78261316516523, '23636': 336.53231842675115, '5290': 337.0188118591025, '392': 338.4966767644402, '6390': 338.5436807124964, '4893': 338.7814342115088, '3895': 339.41794386646114, '9552': 339.6682386048752, '10206': 340.1523741299138, '3162': 340.317665350648, '2523': 344.3086609722156, '58497': 344.9909956215676, '91949': 345.42341138392425, '10782': 345.564047376674, '23659': 345.5706938477936, '7159': 345.6416013209967, '66008': 346.09518684976877, '10797': 347.8499837773991, '701': 348.34548158818285, '6182': 348.5589557106296, '23386': 348.6008679926414, '7398': 348.6161429907747, '2048': 349.1958412170007, '6804': 349.4026688683279, '23224': 349.4488697141463, '9801': 349.47786490533775, '4690': 349.7638013060627, '6117': 350.66823860487517, '3028': 350.88171272732194, '200081': 351.5538640445852, '55556': 352.45476448242846, '1950': 354.33294034878116, '3628': 354.64160132099676, '2184': 355.19037369953753, '11073': 355.5538640445853, '2956': 355.8766210612776, '57215': 355.9353631464977, '836': 356.8080714953873, '22794': 357.44576010399607, '9653': 358.13709913178053, '2597': 360.75361797397386, '998': 362.29611973281385, '8720': 362.50686009652907, '835': 365.82098858620776, '2886': 365.98199124313516, '7106': 367.2048455954331, '10670': 367.992174575224, '9897': 368.2197447421476, '10921': 368.40186576609017, '5048': 369.3760315844494, '6275': 369.98081228947876, '80212': 371.12102103140955, '2064': 373.8464469164298, '3988': 373.8809096250843, '90861': 373.8828916809784, '7077': 373.8969877254553, '84722': 374.16255746200255, '5986': 374.4766859516813, '55825': 375.6584311242051, '873': 377.59186361420905, '3280': 377.6787977883828, '51170': 378.2162078811783, '57178': 378.465323665936, '8851': 378.7990671169549, '3454': 378.95026229321223, '10398': 379.3936644898955, '9261': 379.5973311316721, '63933': 380.747347354273, '29083': 380.861721914563, '9170': 381.9020793914996, '9467': 382.73678817076546, '7048': 383.003912712388, '5289': 383.3646692987041, '23338': 385.19466226334424, '7982': 386.5111486603358, '85236': 386.8523416847119, '9276': 386.89698772545523, '8550': 388.26321182923454, '10898': 388.42303553250554, '7099': 390.1081039405892, '3398': 391.1284706047668, '55008': 392.61142717614894, '11031': 393.7626223524063, '4998': 394.4285030499687, '4331': 394.79043858994123, '11261': 397.2275701669236, '25874': 397.2710372540105, '11344': 397.99726624126845, '7866': 398.79435130232923, '84617': 399.1323833171548, '178': 402.4285030499687, '23585': 402.4355767719072, '10695': 403.82608025225215, '9053': 404.05054080902545, '55012': 404.8209885862077, '6714': 405.73325130979623, '1956': 406.23539559169956}
gene_expression_filter = np.array(gene_expression[list(selected_gene.keys())],dtype="float32")
cell_viability = np.array(cell_viability,dtype="float32")
dtest = xgb.DMatrix(gene_expression_filter,cell_viability)

#Loading model
loaded_model = pickle.load(open("model_adjust\Achilles-L1000-120h_XGBoost_1.dat","rb"))
print("Model Loaded Successfully!")

#Predicting the cell viability
cl_prediction = loaded_model.predict(dtest)
print('Test_Best_Pearson:',sc.stats.pearsonr(cl_prediction,cell_viability))
print("MSE_Test: %.3f" % (mean_squared_error(cl_prediction,cell_viability)))

Model Loaded Successfully!
Test_Best_Pearson: (0.13294886, 1.4938665838889152e-42)
MSE_Test: 2.019
